# Water age in the Terminos lagoon. Mexico

 Jose-Gilberto Cardoso-Mohedano<sup>2,*</sup>, 
 
Lalo
Marisol
María- Eugenia Allende-Arandía 
Carlos 
 
 Joan Albert Sanchez-Cabeza<sup>3</sup>,
 
Ana Carolina Ruiz-Fernández<sup> 3 </sup>, Mario Alejandro Gómez-Ponce <sup>2</sup> ,



<sup> 1 </sup> Posgrado en Ciencias del Mar y Limnología, Universidad Nacional Autónoma de México; Avenida Ciudad Universitaria 3000, C.P. 04510, Coyoacán, Ciudad de México, México

<sup> 2 </sup>  Estación el Carmen, Instituto de Ciencias del Mar y Limnología, Universidad Nacional Autónoma de México, Carretera Carmen-Puerto Real km. 9.5, 24157 Ciudad del Carmen, Campeche, México

<sup> 3 </sup>  Unidad Académica Mazatlán, Instituto de Ciencias del Mar y Limnología, Universidad Nacional Autónoma de México, Calzada Joel Montes Camarena s/n, Colonia Playa Sur, Mazatlán 82000, Sinaloa, México



 \*Corresponding Author: gcardoso@cmarl.unam.mx Phone:+52-938-38-31847

[doi:XXXXX](https://XXXX)

## This scrips evalute Terminos Lagoon Delft3D model performance, using salinity and temperature data field

In [70]:
# Importing libraries
import pandas as pd
import numpy as np
import gsw as gsw
import matplotlib.dates as mdates

In [71]:
# read in the csv file
def read_csv_file(filename):
    # read in the csv file
    df = pd.read_csv(filename, sep=",", header=0, decimal=".", encoding="utf-8") 
    return df

In [72]:
# set folder and file names
sal_temp_terminos_file = "..\data\salinity_temperature_climatological_year_terminos_lagoon.csv"


In [73]:
# Read salinity and temperature data from csv file
sal_temp_terminos_rawdata = read_csv_file(sal_temp_terminos_file)

In [74]:
# print the column names
for col in sal_temp_terminos_rawdata.columns:
    print(col)

Time_model
salinity_ppt_marina_layer1
temperature_C_marina_layer1
Salinity_psu_model
Temperature_C_model
Time _2016_data
Date_GMT_00
Conductivity_microsiemens_cm
Temp_C
Year_2016
Date_GMT_00.1
Conductivity_microsiemens_cm.1
Temp_C.1
Year_2017
Date_GMT_00.2
Conductivity_microsiemens_cm.2
Temp_C.2
Year_2018
Conductivity_microsiemens_cm_average
Conductivity_microsiemens_cm_std
Temp_C_average
Temp_C_std


In [75]:
# select the columns of interest
terminos_sal_temp = sal_temp_terminos_rawdata.copy()[
    [
        "Time_model",
        "salinity_ppt_marina_layer1",
        "temperature_C_marina_layer1",
        "Conductivity_microsiemens_cm_average",
        "Temp_C_average",
    ]
]

In [76]:
# set time as index
terminos_sal_temp["Time_model"] = pd.to_datetime(
    terminos_sal_temp["Time_model"], format="%d/%m/%Y %H:%M"
    )

terminos_sal_temp = terminos_sal_temp.set_index(
    terminos_sal_temp["Time_model"]
    )

In [77]:
months = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]

In [78]:
# Calculate salinity from hobo conductivity

terminos_sal_temp["sea_level_preasure"] = 0

terminos_sal_temp["salinity_psu"] = gsw.conversions.SP_from_C(
    terminos_sal_temp["Conductivity_microsiemens_cm_average"] / 1000,
    terminos_sal_temp["Temp_C_average"],
    terminos_sal_temp["sea_level_preasure"],
)

## Model performance functions

In [79]:
# Calculate basic statistics
def bias(predictions, targets):
    difference = predictions - targets
    bias_val = difference.mean()
    return bias_val

In [80]:
# Mean Square Error (MSE)
def mse(predictions, targets):  
    differences_squared = (predictions - targets) ** 2
    mse_val = differences_squared.mean()
    return mse_val

In [81]:
# Root Mean Square Error (RMSE)
def rmse(predictions, targets):  
    differences_squared = (predictions - targets) ** 2
    rmse_val = np.sqrt(differences_squared.mean())
    return rmse_val

In [82]:
# Mean Absolute Error (MAE)
def mae(predictions, targets):  
    absoluteDifference = np.absolute(predictions - targets)
    mae_val = absoluteDifference.mean()
    return mae_val

In [83]:
# Absolute Maximun Error (AME)
def ame(predictions, targets):  
    difference = predictions - targets
    ame_val = max(np.abs(difference))
    return ame_val

In [84]:
# Model skill score (skill)
def model_skill(predictions, targets):
    differences = predictions - targets
    differencesSquared = abs(differences) ** 2
    sumDifferencesSquared = sum(differencesSquared)
    meanObservations = np.mean(targets)
    modelMinusMeanObservations = np.absolute(list(np.asarray(predictions) - meanObservations))
    observationsMinusMeanObservations = np.absolute(
        list(np.asarray(targets) - meanObservations)
    )
    denominator = sum(
        np.square(modelMinusMeanObservations + observationsMinusMeanObservations)
    )
    skill = 1 - sumDifferencesSquared / denominator
    return skill

In [85]:
# Willmott agreement index (ia)
def willmott_agreement(predictions, targets):
    """
        index of agreement Willmott (1981, 1982) 
        input:
        s: simulated
        o: observed
    output:
        ia: index of agreement
    """
    ia = 1 - (np.sum((targets - predictions) ** 2)) / (
        np.sum(
            (
                np.abs(predictions - np.mean(targets))
                + np.abs(targets - np.mean(targets))
            )
            ** 2
        )
    )
    return ia


In [86]:
def run_performance(predictions, targets):
    from tabulate import tabulate

    performance_metrics = {
        "Bias": bias,
        "Mean Square Error": mse,
        "Root Mean Square Error": rmse,
        "Mean Absolute Error": mae,
        "Absolute Maximum Error": ame,
        "Model Skill": model_skill,
        "Willmott Agreement": willmott_agreement
    }

    performance_results = [[name, round(func(predictions, targets), 1)] for name, func in performance_metrics.items()]
    
    table = tabulate(performance_results, headers=["Analysis", "Results"], tablefmt="simple_outline")
    print(table)

In [87]:
# Salinity model performance analysis
print("Salinity model performance analysis \n ")
run_performance(terminos_sal_temp["salinity_ppt_marina_layer1"], terminos_sal_temp["salinity_psu"])

Salinity model performance analysis 
 
┌────────────────────────┬───────────┐
│ Analysis               │   Results │
├────────────────────────┼───────────┤
│ Bias                   │       0.6 │
│ Mean Square Error      │      15.5 │
│ Root Mean Square Error │       3.9 │
│ Mean Absolute Error    │       3.2 │
│ Absolute Maximum Error │      15.5 │
│ Model Skill            │       0.7 │
│ Willmott Agreement     │       0.7 │
└────────────────────────┴───────────┘


In [88]:
print("Temperature model performance analysis\n")
run_performance(terminos_sal_temp["temperature_C_marina_layer1"], terminos_sal_temp["Temp_C_average"])

Temperature model performance analysis

┌────────────────────────┬───────────┐
│ Analysis               │   Results │
├────────────────────────┼───────────┤
│ Bias                   │      -1.8 │
│ Mean Square Error      │       4.9 │
│ Root Mean Square Error │       2.2 │
│ Mean Absolute Error    │       1.9 │
│ Absolute Maximum Error │       5.9 │
│ Model Skill            │       0.8 │
│ Willmott Agreement     │       0.8 │
└────────────────────────┴───────────┘
